# Merge Metadata

- pull from downloaded google sheets of FAIRe terms

### Import

In [75]:
import pandas as pd
import numpy as np
import glob

### Functions

In [76]:
def water_amplicon_merge(water_df, amplicon_df):
    df = water_df.set_index('sample_name')
    df = df.drop(['sheet'], axis=1)
    # add amplicon metadata to water sample metadata (some water samples don't have amplicon data if not sequenced in institute)
    df = df.join(amplicon_df.set_index('sample_name'), rsuffix='_amplicon_sheet')
    df = df.drop(['sheet'], axis=1)
    return df
def study_data_merge(study_meta, bioinf_meta, institute_name):
    df = pd.concat([study_meta, bioinf_meta], axis=1)
    df = df.drop(['sheet', 'date_modified'], axis=1)
    df = df.T
    df = df.rename(columns={0:institute_name})
    return df
    

### Metadata Directory

In [77]:
meta_dir = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/'
files = glob.glob(meta_dir + '*.xlsx')
print(files)

['/Users/kpitz/github/GLOMICON/intercomparison/metadata/SBR_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx', '/Users/kpitz/github/GLOMICON/intercomparison/metadata/~$AWI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx', '/Users/kpitz/github/GLOMICON/intercomparison/metadata/MBARI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx', '/Users/kpitz/github/GLOMICON/intercomparison/metadata/AWI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx', '/Users/kpitz/github/GLOMICON/intercomparison/metadata/NOAA_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx']


## Import and Merge Metadata

In [78]:
study_dfs = []
samp_dfs = []
institutes = []

### MBARI

In [262]:
# import
#file = files[1]
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/MBARI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
print(file)
inst_name = 'MBARI'
institutes.append(inst_name)
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,6]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

/Users/kpitz/github/GLOMICON/intercomparison/metadata/MBARI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx
Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 6


/Users/kpitz/anaconda3/envs/scipy_test/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,amplicon_sequenced,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,date_modified,sheet
0,18S V9,atropos,dada2,ASV,BLAST; MEGAN LCA,genbank NR,https://github.com/MBARI-BOG/BOG-Banzai-Dada2-...,2024-08-15 08:44:22.928,analysis_data


In [263]:
# get columns in water sample metadata
water_samp_cols = list(dfs[1])
amplicon_samp_cols = list(dfs[2])

In [80]:
df = study_data_merge(dfs[0], dfs[3], inst_name)
study_dfs.append(df)
df.head()

,MBARI
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid...."
project_description,"Replicate filters, originating from surface wa..."


In [81]:
df = water_amplicon_merge(dfs[1], dfs[2])
samp_dfs.append(df)
df.head()

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,project_id,...,target_gene,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified_amplicon_sheet,modified_by_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,,,
CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-08-15 08:42:28.469,NaN
CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-06-18 19:31:14.358,kpitz@mbari.org
CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN
CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN
CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN


### AOML

In [82]:
# import
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/NOAA_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
print(file)
inst_name = 'AOML'
institutes.append(inst_name)
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,6]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

/Users/kpitz/github/GLOMICON/intercomparison/metadata/NOAA_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx
Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 6


,amplicon_sequenced,ampliconSize,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,identificationReferences,controls_used,date_modified,modified_by,sheet
0,16S V4-V5,411,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,Silva SSU Ref NR 99 v138.1; 515f-926r region; ...,https://github.com/aomlomics/gomecc,10.5281/zenodo.8392695 | https://github.com/ao...,12 distilled water blanks | 2 PCR no-template ...,2024-04-11 07:12:09.354,aomlomics@gmail.com,analysis_data
1,18S V9,260,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,PR2 v5.0.1; V9 1391f-1510r region; 10.5281/zen...,https://github.com/aomlomics/gomecc,10.5281/zenodo.8392706 | https://pr2-database....,12 distilled water blanks | 2 PCR no-template ...,NaT,NaN,analysis_data


In [83]:
df = study_data_merge(dfs[0], dfs[3], inst_name)
df = df.rename(columns={1:'AOML_2'})
study_dfs.append(df)
df.head(n=50)

,AOML,AOML_2
project_id,GLOMICON_INTERCOMP,NaN
project_name,Intercomparison of filtered seawater samples a...,NaN
project_id_external,GLOMICON_INTERCOMP,NaN
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN
type,Occurrence,NaN
license,CC-BY 4.0,NaN
external_links,https://github.com/GLOMICON,NaN
amplicon_sequenced,16S V4-V5,18S V9
ampliconSize,411,260
trim_method,cutadapt,cutadapt


In [84]:
df = water_amplicon_merge(dfs[1], dfs[2])
samp_dfs.append(df)
df.head(n=50)

,Collecting Institute,Analyzing Institute,description,date_modified,collecting_institute,analyzing_Institute,library_id,library_strategy,library_source,library_selection,lib_layout,platform,instrument_model,filetype,filename,filename2,sequencing_run_folder,pcr_primer_name_reverse,date_modified_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,
GLOMICON_AWI_12,AWI,NOAA,2024-09-26 09:18:08.071,2024-09-26 09:18:37.306,AWI,NOAA,GLOMICON_AWI_12,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_AWI_12_S69_L001_R1_001.fastq.gz,GLOMICON_AWI_12_S69_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,2024-09-12 20:02:53.389
GLOMICON_AWI_16,AWI,NOAA,NaT,NaT,AWI,NOAA,GLOMICON_AWI_16,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_AWI_16_S68_L001_R1_001.fastq.gz,GLOMICON_AWI_16_S68_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_AWI_20,AWI,NOAA,NaT,NaT,AWI,NOAA,GLOMICON_AWI_20,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_AWI_20_S67_L001_R1_001.fastq.gz,GLOMICON_AWI_20_S67_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_AWI_4,AWI,NOAA,NaT,NaT,AWI,NOAA,GLOMICON_AWI_4,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_AWI_4_S71_L001_R1_001.fastq.gz,GLOMICON_AWI_4_S71_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_AWI_8,AWI,NOAA,NaT,NaT,AWI,NOAA,GLOMICON_AWI_8,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_AWI_8_S70_L001_R1_001.fastq.gz,GLOMICON_AWI_8_S70_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_BloomMock_1,MOCKBLOOM,NOAA,NaT,NaT,MOCKBLOOM,NOAA,GLOMICON_BloomMock_1,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_BloomMock_1_S77_L001_R1_001.fastq.gz,GLOMICON_BloomMock_1_S77_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_BloomMock_10,MOCKBLOOM,NOAA,NaT,NaT,MOCKBLOOM,NOAA,GLOMICON_BloomMock_10,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_BloomMock_10_S80_L001_R1_001.fastq.gz,GLOMICON_BloomMock_10_S80_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_BloomMock_30,MOCKBLOOM,NOAA,NaT,NaT,MOCKBLOOM,NOAA,GLOMICON_BloomMock_30,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_BloomMock_30_S81_L001_R1_001.fastq.gz,GLOMICON_BloomMock_30_S81_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,NaN,NaT
GLOMICON_BloomMock_4,MOCKBLOOM,NOAA,NaT,NaT,MOCKBLOOM,NOAA,GLOMICON_BloomMock_4,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,GLOMICON_BloomMock_4_S78_L001_R1_001.fastq.gz,GLOMICON_BloomMock_4_S78_L001_R2_001.fastq.gz,20240401_18S-Amplicon_PE150,x,NaT


### AWI

In [85]:
# import
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/AWI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
print(file)
inst_name = 'AWI'
institutes.append(inst_name)
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,8]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

/Users/kpitz/github/GLOMICON/intercomparison/metadata/AWI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx
Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 8


,amplicon_sequenced,ampliconSize,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,identificationReferences,controls_used,date_modified,modified_by,sheet
0,18S V4,356-394,cutadapt v2.8,dada denoising; dada2 v1.30 in R v4.3.2 (learn...,ASV,dada2 implementation of naive-bayes classifier...,PR2 v5.0.0; full set; https://github.com/pr2da...,???,https://github.com/pr2database/pr2database/rel...,???,2024-09-06 09:12:48.224,stefan.neuhaus@awi.de,analysis_data


In [86]:
df = study_data_merge(dfs[0], dfs[3], inst_name)
df = df.rename(columns={1:inst_name+'_2'})
study_dfs.append(df)
df.head(n=50)

,AWI
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid...."
project_description,"Replicate filters, originating from surface wa..."
type,Occurrence
license,CC-BY 4.0
associated_parties,"Felix Janssen, felix.janssen@awi.de, https://o..."
study_area_description,Surface waters from different marine time seri...
external_links,https://github.com/GLOMICON


In [87]:
df = water_amplicon_merge(dfs[1], dfs[2])
samp_dfs.append(df)
df.head(n=50)

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,sample_title,notes_sampling,project_id,...,amplicon_sequenced,target_gene,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,,,
Samples taken / prepared and analyzed by AWI (complete information),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
Framstrait_01,AWI,AWI,PS114 (2018),PS114-36-4,pooled,01,GLOMICON_INTERCOMP_Framstrait_01,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_01,AWI,AWI,PS114 (2018),PS114-36-4,pooled,01,GLOMICON_INTERCOMP_Framstrait_01,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_05,AWI,AWI,PS114 (2018),PS114-36-4,pooled,05,GLOMICON_INTERCOMP_Framstrait_05,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_05,AWI,AWI,PS114 (2018),PS114-36-4,pooled,05,GLOMICON_INTERCOMP_Framstrait_05,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_09,AWI,AWI,PS114 (2018),PS114-36-4,pooled,09,GLOMICON_INTERCOMP_Framstrait_09,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_09,AWI,AWI,PS114 (2018),PS114-36-4,pooled,09,GLOMICON_INTERCOMP_Framstrait_09,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_13,AWI,AWI,PS114 (2018),PS114-36-4,pooled,13,GLOMICON_INTERCOMP_Framstrait_13,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT
Framstrait_13,AWI,AWI,PS114 (2018),PS114-36-4,pooled,13,GLOMICON_INTERCOMP_Framstrait_13,Seawater sample Fram Strait (GLOMICON_INTERCOMP),Surface waters from the northern part of Fram ...,GLOMICON_INTERCOMP,...,18S V4,18S rRNA,V4,5'-GCG GTA ATT CCA GCT CCA A-3',5'-AC TTT CGT TCT TGA TYR R-3',528i F,964i R,528F: Elwood et al. (1985) doi:10.1093/oxfordj...,\ninitial denaturation:95degC_3min;denaturatio...,NaT


### SBR

In [244]:
# import
#file = files[1]
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/SBR_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
print(file)
inst_name = 'SBR'
institutes.append(inst_name)
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,6]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

/Users/kpitz/github/GLOMICON/intercomparison/metadata/SBR_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx
Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 6


,amplicon_sequenced,ampliconSize,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,identificationReferences,controls_used,date_modified,modified_by,sheet
0,16S V4-V5,411,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,Silva SSU Ref NR 99 v138.1; 515f-926r region; ...,https://github.com/aomlomics/gomecc,10.5281/zenodo.8392695 | https://github.com/ao...,12 distilled water blanks | 2 PCR no-template ...,2024-04-11 07:12:09.354,aomlomics@gmail.com,analysis_data
1,18S V9,260,cutadapt,Tourmaline; qiime2-2021.2; dada2,ASV,Tourmaline; qiime2-2021.2; naive-bayes classifier,PR2 v5.0.1; V9 1391f-1510r region; 10.5281/zen...,https://github.com/aomlomics/gomecc,10.5281/zenodo.8392706 | https://pr2-database....,12 distilled water blanks | 2 PCR no-template ...,NaT,NaN,analysis_data


In [89]:
df = study_data_merge(dfs[0], dfs[3], inst_name)
df = df.rename(columns={1:inst_name+'_2'})
study_dfs.append(df)
df.head()

,SBR,SBR_2
project_id,GLOMICON_INTERCOMP,NaN
project_name,Intercomparison of filtered seawater samples a...,NaN
project_id_external,GLOMICON_INTERCOMP,NaN
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN
project_description,"Replicate filters, originating from surface wa...",NaN


In [90]:
df = water_amplicon_merge(dfs[1], dfs[2])
samp_dfs.append(df)
df.head()

,Collecting Institute,Analyzing Institute,station,sample_replicate,source_material_id,project_id,organism,collection_date_local,collection_date,depth,...,amplicon_sequenced,target_gene,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,,,
Samples taken / prepared and analyzed by SBR (complete information),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
SBR01,SBR,SBR,SOMLIT-ASTAN,1.0,GLOMICON_INTERCOMP_SBR_01,GLOMICON_INTERCOMP,seawater metagenome,2018-10-17,2018-10-17,3 m,...,18S V4,18S rRNA,V4,5’-CCAGCASCYGCGGTAATTCC-3’,5’-ACTTTCGTTCTTGATYRA-3’,TAReuk454FWD1,TAReukREV3,TAReuk454FWD1 (5’-CCAGCASCYGCGGTAATTCC-3’) and...,"initial denaturation at 98°C for 30 seconds, f...",2024-09-17 05:03:26.170
SBR05,SBR,SBR,SOMLIT-ASTAN,5.0,GLOMICON_INTERCOMP_SBR_05,GLOMICON_INTERCOMP,seawater metagenome,2018-10-17,2018-10-17,3 m,...,18S V4,18S rRNA,V4,5’-CCAGCASCYGCGGTAATTCC-3’,5’-ACTTTCGTTCTTGATYRA-3’,TAReuk454FWD1,TAReukREV3,TAReuk454FWD1 (5’-CCAGCASCYGCGGTAATTCC-3’) and...,"initial denaturation at 98°C for 30 seconds, f...",NaT
SBR09,SBR,SBR,SOMLIT-ASTAN,9.0,GLOMICON_INTERCOMP_SBR_09,GLOMICON_INTERCOMP,seawater metagenome,2018-10-17,2018-10-17,3 m,...,18S V4,18S rRNA,V4,5’-CCAGCASCYGCGGTAATTCC-3’,5’-ACTTTCGTTCTTGATYRA-3’,TAReuk454FWD1,TAReukREV3,TAReuk454FWD1 (5’-CCAGCASCYGCGGTAATTCC-3’) and...,"initial denaturation at 98°C for 30 seconds, f...",NaT
SBR13,SBR,SBR,SOMLIT-ASTAN,13.0,GLOMICON_INTERCOMP_SBR_13,GLOMICON_INTERCOMP,seawater metagenome,2018-10-17,2018-10-17,3 m,...,18S V4,18S rRNA,V4,5’-CCAGCASCYGCGGTAATTCC-3’,5’-ACTTTCGTTCTTGATYRA-3’,TAReuk454FWD1,TAReukREV3,TAReuk454FWD1 (5’-CCAGCASCYGCGGTAATTCC-3’) and...,"initial denaturation at 98°C for 30 seconds, f...",NaT


## Merge across institutes

In [95]:
# 04/28/25 - looks like SBR didn't alter analysis sheet, sample as AOML
# study data
df = pd.concat(study_dfs, axis=1)
# export
file = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/metadata/' + 'combined_study_data.csv'
df.to_csv(file)
df

,MBARI,AOML,AOML_2,AWI,SBR,SBR_2
project_id,GLOMICON_INTERCOMP,GLOMICON_INTERCOMP,NaN,GLOMICON_INTERCOMP,GLOMICON_INTERCOMP,NaN
project_name,Intercomparison of filtered seawater samples a...,Intercomparison of filtered seawater samples a...,NaN,Intercomparison of filtered seawater samples a...,Intercomparison of filtered seawater samples a...,NaN
project_id_external,GLOMICON_INTERCOMP,GLOMICON_INTERCOMP,NaN,GLOMICON_INTERCOMP,GLOMICON_INTERCOMP,NaN
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid....","Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN,"Kathleen Pitz, kpitz@mbari.org, https://orcid....","Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN
project_description,"Replicate filters, originating from surface wa...",NaN,NaN,"Replicate filters, originating from surface wa...","Replicate filters, originating from surface wa...",NaN
type,Occurrence,Occurrence,NaN,Occurrence,Occurrence,NaN
license,CC-BY 4.0,CC-BY 4.0,NaN,CC-BY 4.0,CC-BY 4.0,NaN
associated_parties,"Francisco Chavez, chfr@mbari.org",NaN,NaN,"Felix Janssen, felix.janssen@awi.de, https://o...","Christian Jeanthon <jeanthon@sb-roscoff.fr>, C...",NaN
study_area_description,eDNA from north of Monterey Bay collected duri...,NaN,NaN,Surface waters from different marine time seri...,Surface waters from different marine time seri...,NaN
amplicon_sequenced,18S V9,16S V4-V5,18S V9,18S V4,16S V4-V5,18S V9


In [94]:
# sample data
df = pd.concat(samp_dfs, axis=0, keys=institutes)
# export
file = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/metadata/' + 'combined_sample_data.csv'
df.to_csv(file)
df

Collecting Institute  \
      sample_name                                                           
MBARI CN18Sc37_12_Rep_Stdy5_AO                                      MBARI   
      CN18Sc37_12_Rep_Stdy10_AO                                     MBARI   
      CN18Sc37_12_Rep_Stdy14_AO                                     MBARI   
      CN18Sc37_12_Rep_Stdy18_AO                                     MBARI   
      CN18Sc37_12_Rep_Stdy22_AO                                     MBARI   
...                                                                   ...   
SBR   [to be added from GLOMICON partner's metadata]                  SBR   
      [to be added from GLOMICON partner's metadata]                  SBR   
      [to be added from GLOMICON partner's metadata]                  SBR   
      [to be added from GLOMICON partner's metadata]                  SBR   
      NaN                                                             NaN   

                                                     Analyzing Institute  \
      sample_name                                                          
MBARI CN18Sc37_12_Rep_Stdy5_AO                                     MBARI   
      CN18Sc37_12_Rep_Stdy10_AO                                    MBARI   
      CN18Sc37_12_Rep_Stdy14_AO                                    MBARI   
      CN18Sc37_12_Rep_Stdy18_AO                                    MBARI   
      CN18Sc37_12_Rep_Stdy22_AO                                    MBARI   
...                                                                  ...   
SBR   [to be added from GLOMICON partner's metadata]          UDalhousie   
      [to be added from GLOMICON partner's metadata]          UDalhousie   
      [to be added from GLOMICON partner's metadata]          UDalhousie   
      [to be added from GLOMICON partner's metadata]          UDalhousie   
      NaN                                                            NaN   

                                                     cruise_id       station  \
      sample_name                                                              
MBARI CN18Sc37_12_Rep_Stdy5_AO                           CN18S            TS   
      CN18Sc37_12_Rep_Stdy10_AO                          CN18S            TS   
      CN18Sc37_12_Rep_Stdy14_AO                          CN18S            TS   
      CN18Sc37_12_Rep_Stdy18_AO                          CN18S            TS   
      CN18Sc37_12_Rep_Stdy22_AO                          CN18S            TS   
...                                                        ...           ...   
SBR   [to be added from GLOMICON partner's metadata]       NaN  SOMLIT-ASTAN   
      [to be added from GLOMICON partner's metadata]       NaN  SOMLIT-ASTAN   
      [to be added from GLOMICON partner's metadata]       NaN  SOMLIT-ASTAN   
      [to be added from GLOMICON partner's metadata]       NaN  SOMLIT-ASTAN   
      NaN                                                  NaN           NaN   

                                                     ctd_bottle_no  \
      sample_name                                                    
MBARI CN18Sc37_12_Rep_Stdy5_AO                                12.0   
      CN18Sc37_12_Rep_Stdy10_AO                               12.0   
      CN18Sc37_12_Rep_Stdy14_AO                               12.0   
      CN18Sc37_12_Rep_Stdy18_AO                               12.0   
      CN18Sc37_12_Rep_Stdy22_AO                               12.0   
...                                                            ...   
SBR   [to be added from GLOMICON partner's metadata]           NaN   
      [to be added from GLOMICON partner's metadata]           NaN   
      [to be added from GLOMICON partner's metadata]           NaN   
      [to be added from GLOMICON partner's metadata]           NaN   
      NaN                                                      NaN   

                                                     sample_replicate  \
      sample_name                    

In [273]:
# sample data
df = pd.concat(samp_dfs, axis=0, keys=institutes)
#water_samp_cols.remove('sample_name')
#water_samp_cols.remove('sheet')

df = df[water_samp_cols]
#df = df.loc[df['temp'].isna()==True]

df.tail(n=20)

/var/folders/5v/h4s_0pz17yb6f6krsxkzr5dw000b65/T/ipykernel_4375/2190577957.py:2: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df = pd.concat(samp_dfs, axis=0, keys=institutes)


Collecting Institute  \
    sample_name                                                               
SBR SOC12                                                               NOC   
    SOC18                                                               NOC   
    SOC24                                                               NOC   
    SOC28                                                               NOC   
    DAL03                                                        UDalhousie   
    DAL09                                                        UDalhousie   
    DAL15                                                        UDalhousie   
    DAL21                                                        UDalhousie   
    DAL27                                                        UDalhousie   
    EMk03                                                               AWI   
    EMK08                                                               AWI   
    EMk09                                                               AWI   
    EMk15                                                               AWI   
    EMk17                                                               AWI   
    BMk13                                                               AWI   
    BMk15                                                               AWI   
    BMk17                                                               AWI   
    BMk21                                                               AWI   
    BMk25                                                               AWI   
    Samples taken / prepared by SBR and analyzed by...                  NaN   

                                                       Analyzing Institute  \
    sample_name                                                              
SBR SOC12                                                              SBR   
    SOC18                                                              SBR   
    SOC24                                                              SBR   
    SOC28                                                              SBR   
    DAL03                                                              SBR   
    DAL09                                                              SBR   
    DAL15                                                              SBR   
    DAL21                                                              SBR   
    DAL27                                                              SBR   
    EMk03                                                              SBR   
    EMK08                                                              SBR   
    EMk09                                                              SBR   
    EMk15                                                              SBR   
    EMk17                                                              SBR   
    BMk13                                                              SBR   
    BMk15                                                              SBR   
    BMk17                                                              SBR   
    BMk21                                                              SBR   
    BMk25                                                              SBR   
    Samples taken / prepared by SBR and analyzed by...                 NaN   

                                                       cruise_id station  \
    sample_name                                                            
SBR SOC12                                                    NaN     NaN   
    SOC18                                                    NaN     NaN   
    SOC24                                                    NaN     NaN   
    SOC28                                                    NaN     NaN   
    DAL03                                                    NaN     NaN   
    DAL09                                                    NaN   

In [261]:
# edit sample data
# want to merge metadata from collecting institutions with analyzing institutions with one filter per row (at the end)
# each filter has a unique name that can be made from its collecting origin and its replicate number
# difficulty is different naming between analyzing institutions

# combine all sample data
df = pd.concat(samp_dfs, axis=0, keys=institutes)
df = df.reset_index()

# every sample has a collecting institute and a number associated with it
# drop columns that are all nan
df = df.dropna(how='all', axis=1)

# create new sample number column
df['samp_num'] = np.nan

# get numbers associated with names - can be either in sample_name or in source_material_id columns
# only name with numbers is 'CN18Sc37_12_Rep_Stdy'
df['samp_num'] = df['sample_name'].str.replace('CN18Sc37_12_Rep_Stdy', 'MBARI_').str.extract('(\\d+)')
df.loc[df['samp_num'].isna(),'samp_num' ] = df['source_material_id'].str.replace('CN18Sc37_12_Rep_Stdy', 'MBARI_').str.extract('(\\d+)')[0]

# create unified short names, corresponding with origin:
df['short_name'] = ''
print(df['Collecting Institute'].unique())
# ['MBARI' 'NOAA' 'MOCKBLOOM' 'NOC' 'UDalhousie' 'MOCKEVEN' 'AWI' 'SBR' nan]
df['short_name'] = df['Collecting Institute']
# take care of controls:
df.loc[df['sample_name'].str.contains('evenMock|emk|MOCKEVEN', case=False)==True, 'short_name'] = 'evenMock'
df.loc[df['sample_name'].str.contains('bloomMock|bmk|MOCKBLOOM', case=False)==True, 'short_name'] = 'bloomMock'
# if no sample_name ...
df.loc[df['source_material_id'].str.contains('evenMock|emk', case=False)==True, 'short_name'] = 'evenMock'
df.loc[df['source_material_id'].str.contains('bloomMock|bmk', case=False)==True, 'short_name'] = 'bloomMock'

#blank_01 - a single blank AWI ran and included, don't want to mix it up with mock communities.
df.loc[df['sample_name'].str.contains('blank_01', case=False)==True, 'short_name'] = 'blank'
print('short_name',df['short_name'].unique())
df['unified_name'] = df['short_name'] + '_' + df['samp_num'].astype(float).astype(str)
df = df.loc[df['unified_name'].isna()==False]
df['samp_num'] = df['samp_num'].astype(float)

df['sample_name'] = df['sample_name'].str.replace("[to be added from GLOMICON partner's metadata]",'')

'''
df['sample_name'] = df['sample_name'].str.replace("[to be added from GLOMICON partner's metadata]",'')

new_samp_dfs = []
new_samp_names = []
new_keys = []
for i,group in df.groupby('level_0'):
    group = group.drop_duplicates('unified_name')
    #gruop = group.dropna(how='all', axis=1)
    group.set_index(['short_name', 'samp_num','unified_name'], inplace=True)
    new_samp_dfs.append(group)
    new_samp_names.append(group[['sample_name', 'source_material_id']])
    new_keys.append(i)
    
'''
# want one row per sequenced sample
cols = ['level_0', 'unified_name', 'sample_name', 'Collecting Institute', 'Analyzing Institute', 'short_name', 'samp_num', 'source_material_id', 'dna_conc', 'size_frac', 'salinity', 'amplicon_sequenced']
df = df[cols]

# for now remove mock controls
df = df.loc[df['short_name'].isin(['bloomMock', 'evenMock'])==False]
# subset just to make it work for one group...
df = df.loc[df['short_name'].isin(['MBARI'])==True]



df['seq'] = ''
df.loc[df['sample_name'].isna()==False, 'seq'] = 'sequenced'
df.loc[df['seq']=='', 'Analyzing Institute'] = np.nan

#df = df.loc[df['amplicon_sequenced'].isna()==True] # get all unsequenced samples

#df = df.groupby('unified_name').sum()

df.head(n=50)

['MBARI' 'NOAA' 'MOCKBLOOM' 'NOC' 'UDalhousie' 'MOCKEVEN' 'AWI' 'SBR' nan]
short_name ['MBARI' 'NOAA' 'bloomMock' 'NOC' 'UDalhousie' 'evenMock' 'AWI' 'SBR' nan
 'blank']


/var/folders/5v/h4s_0pz17yb6f6krsxkzr5dw000b65/T/ipykernel_4375/801486836.py:7: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df = pd.concat(samp_dfs, axis=0, keys=institutes)


,level_0,unified_name,sample_name,Collecting Institute,Analyzing Institute,short_name,samp_num,source_material_id,dna_conc,size_frac,salinity,amplicon_sequenced,seq
0,MBARI,MBARI_5.0,CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,MBARI,5.0,CN18Sc37_12_Rep_Stdy5,NaN,0.22um,33.5385,18S V9,sequenced
1,MBARI,MBARI_10.0,CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,MBARI,10.0,CN18Sc37_12_Rep_Stdy10,NaN,0.22um,33.5385,18S V9,sequenced
2,MBARI,MBARI_14.0,CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,MBARI,14.0,CN18Sc37_12_Rep_Stdy14,NaN,0.22um,33.5385,18S V9,sequenced
3,MBARI,MBARI_18.0,CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,MBARI,18.0,CN18Sc37_12_Rep_Stdy18,NaN,0.22um,33.5385,18S V9,sequenced
4,MBARI,MBARI_22.0,CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,MBARI,22.0,CN18Sc37_12_Rep_Stdy22,NaN,0.22um,33.5385,18S V9,sequenced
28,MBARI,MBARI_1.0,NaN,MBARI,NaN,MBARI,1.0,CN18Sc37_12_Rep_Stdy1,NaN,0.22um,33.5385,NaN,
29,MBARI,MBARI_2.0,NaN,MBARI,NaN,MBARI,2.0,CN18Sc37_12_Rep_Stdy2,NaN,0.22um,33.5385,NaN,
30,MBARI,MBARI_3.0,NaN,MBARI,NaN,MBARI,3.0,CN18Sc37_12_Rep_Stdy3,NaN,0.22um,33.5385,NaN,
31,MBARI,MBARI_4.0,NaN,MBARI,NaN,MBARI,4.0,CN18Sc37_12_Rep_Stdy4,NaN,0.22um,33.5385,NaN,
32,MBARI,MBARI_6.0,NaN,MBARI,NaN,MBARI,6.0,CN18Sc37_12_Rep_Stdy6,NaN,0.22um,33.5385,NaN,


In [216]:
df = pd.concat(new_samp_dfs, axis=1, keys = new_keys)
#df = pd.concat(new_samp_dfs, axis=1)
df = df.dropna(how='all', axis=1)
df = df.sort_index()
df.to_csv('/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/metadata/test.csv')

df = df.loc[:, (slice(None), 'sample_name')]


df

AOML  \
                                           sample_name   
short_name samp_num unified_name                         
AWI        1.0      AWI_1.0                        NaN   
           2.0      AWI_2.0                        NaN   
           3.0      AWI_3.0                        NaN   
           4.0      AWI_4.0             GLOMICON_AWI_4   
           5.0      AWI_5.0                        NaN   
...                                                ...   
evenMock   15.0     evenMock_15.0                  NaN   
           17.0     evenMock_17.0                  NaN   
           18.0     evenMock_18.0                  NaN   
           20.0     evenMock_20.0                  NaN   
           NaN      evenMock_nan   GLOMICON_EvenMock_A   

                                                                              AWI  \
                                                                      sample_name   
short_name samp_num unified_name                                                    
AWI        1.0      AWI_1.0                                         Framstrait_01   
           2.0      AWI_2.0        [to be added from GLOMICON partner's metadata]   
           3.0      AWI_3.0        [to be added from GLOMICON partner's metadata]   
           4.0      AWI_4.0        [to be added from GLOMICON partner's metadata]   
           5.0      AWI_5.0                                         Framstrait_05   
...                                                                           ...   
evenMock   15.0     evenMock_15.0                                           EMk15   
           17.0     evenMock_17.0                                           EMk17   
           18.0     evenMock_18.0  [to be added from GLOMICON partner's metadata]   
           20.0     evenMock_20.0  [to be added from GLOMICON partner's metadata]   
           NaN      evenMock_nan   [to be added from GLOMICON partner's metadata]   

                                        MBARI         SBR  
                                  sample_name sample_name  
short_name samp_num unified_name                           
AWI        1.0      AWI_1.0               NaN         NaN  
           2.0      AWI_2.0               NaN         NaN  
           3.0      AWI_3.0               NaN         NaN  
           4.0      AWI_4.0               NaN         NaN  
           5.0      AWI_5.0               NaN         NaN  
...                                       ...         ...  
evenMock   15.0     evenMock_15.0         NaN       EMk15  
           17.0     evenMock_17.0         NaN       EMk17  
           18.0     evenMock_18.0         NaN         NaN  
           20.0     evenMock_20.0         NaN         NaN  
           NaN      evenMock_nan          NaN         NaN  

[150 rows x 4 columns]

In [213]:
df = pd.concat(new_samp_names, axis=0, keys = new_keys)
#df = pd.concat(new_samp_dfs, axis=1)
df = df.dropna(how='all', axis=1)
df = df.sort_index()
df.to_csv('/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/metadata/test.csv')
df

sample_name source_material_id
     short_name samp_num unified_name                                     
AOML AWI        4.0      AWI_4.0         GLOMICON_AWI_4                NaN
                8.0      AWI_8.0         GLOMICON_AWI_8                NaN
                12.0     AWI_12.0       GLOMICON_AWI_12                NaN
                16.0     AWI_16.0       GLOMICON_AWI_16                NaN
                20.0     AWI_20.0       GLOMICON_AWI_20                NaN
...                                                 ...                ...
SBR  evenMock   3.0      evenMock_3.0             EMk03                NaN
                8.0      evenMock_8.0             EMK08                NaN
                9.0      evenMock_9.0             EMk09                NaN
                15.0     evenMock_15.0            EMk15                NaN
                17.0     evenMock_17.0            EMk17                NaN

[197 rows x 2 columns]

In [173]:
# edit sample data
# want to merge metadata from collecting institutions with analyzing institutions with one filter per row (at the end)
# each filter has a unique name that can be made from its collecting origin and its replicate number
# difficulty is different naming between analyzing institutions

# combine all sample data
df = pd.concat(samp_dfs, axis=0, keys=institutes)
df = df.reset_index()

# every sample has a collecting institute and a number associated with it
# drop columns that are all nan
df = df.dropna(how='all', axis=1)

# create new sample number column
df['samp_num'] = np.nan

# get numbers associated with names - can be either in sample_name or in source_material_id columns
# only name with numbers is 'CN18Sc37_12_Rep_Stdy'
df['samp_num'] = df['sample_name'].str.replace('CN18Sc37_12_Rep_Stdy', 'MBARI_').str.extract('(\\d+)')
df.loc[df['samp_num'].isna(),'samp_num' ] = df['source_material_id'].str.replace('CN18Sc37_12_Rep_Stdy', 'MBARI_').str.extract('(\\d+)')[0]

# create unified short names, corresponding with origin:
df['short_name'] = ''
print(df['Collecting Institute'].unique())
# ['MBARI' 'NOAA' 'MOCKBLOOM' 'NOC' 'UDalhousie' 'MOCKEVEN' 'AWI' 'SBR' nan]
df['short_name'] = df['Collecting Institute']
# take care of controls:
df.loc[df['sample_name'].str.contains('evenMock|emk|MOCKEVEN', case=False)==True, 'short_name'] = 'evenMock'
df.loc[df['sample_name'].str.contains('bloomMock|bmk|MOCKBLOOM', case=False)==True, 'short_name'] = 'bloomMock'
# if no sample_name ...
df.loc[df['source_material_id'].str.contains('evenMock|emk', case=False)==True, 'short_name'] = 'evenMock'
df.loc[df['source_material_id'].str.contains('bloomMock|bmk', case=False)==True, 'short_name'] = 'bloomMock'

#blank_01 - a single blank AWI ran and included, don't want to mix it up with mock communities.
df.loc[df['sample_name'].str.contains('blank_01', case=False)==True, 'short_name'] = 'blank'

print('short_name',df['short_name'].unique())

df['unified_name'] = df['short_name'] + '_' + df['samp_num'].astype(float).astype(str)


df = df.loc[df['unified_name'].isna()==False]

cols = ['unified_name','level_0', 'sample_name', 'Collecting Institute', 'Analyzing Institute','samp_num', 'source_material_id', 'organism', 'short_name']
#df = df.loc[df['samp_num']=='']
#df = df.loc[df['Collecting Institute']=='NOAA']
#print(df['Collecting Institute'].unique())
#df = df.loc[df['Collecting Institute'].isin(['AWI', 'MOCKBLOOM', 'MOCKEVEN'])]
#df = df.loc[df['Collecting Institute'].isin([ 'MOCKBLOOM', 'MOCKEVEN'])]

#df = df.sort_values(['short_name', 'samp_num', 'unified_name','Collecting Institute', 'organism'], ascending=False)
df['samp_num'] = df['samp_num'].astype(float)
#df = df.sort_values(['short_name', 'samp_num', 'unified_name','level_0','Collecting Institute', 'organism'], ascending=True)
df = df.sort_values(['level_0','Analyzing Institute','short_name', 'samp_num', 'unified_name','level_0','Collecting Institute', 'organism'], ascending=True)
#df[cols].head(n=50)

#df = df.set_index(['short_name', 'samp_num', 'unified_name' ])

#df = df.pivot(index='unified_name', values='source_material_id', columns='Collecting Institute')
df = df[cols]
df = df.drop_duplicates()  # AWI sequenced same filters twice

#df = df.pivot(index='unified_name', columns='level_0', values='sample_name')
df[cols].to_csv('/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/metadata/test.csv')
df[cols]
df

['MBARI' 'NOAA' 'MOCKBLOOM' 'NOC' 'UDalhousie' 'MOCKEVEN' 'AWI' 'SBR' nan]
short_name ['MBARI' 'NOAA' 'bloomMock' 'NOC' 'UDalhousie' 'evenMock' 'AWI' 'SBR' nan
 'blank']


,unified_name,level_0,sample_name,Collecting Institute,Analyzing Institute,samp_num,source_material_id,organism,short_name
51,AWI_4.0,AOML,GLOMICON_AWI_4,AWI,NOAA,4.0,NaN,NaN,AWI
52,AWI_8.0,AOML,GLOMICON_AWI_8,AWI,NOAA,8.0,NaN,NaN,AWI
48,AWI_12.0,AOML,GLOMICON_AWI_12,AWI,NOAA,12.0,NaN,NaN,AWI
49,AWI_16.0,AOML,GLOMICON_AWI_16,AWI,NOAA,16.0,NaN,NaN,AWI
50,AWI_20.0,AOML,GLOMICON_AWI_20,AWI,NOAA,20.0,NaN,NaN,AWI
...,...,...,...,...,...,...,...,...,...
235,SBR_4.0,SBR,[to be added from GLOMICON partner's metadata],SBR,UDalhousie,4.0,GLOMICON_INTERCOMP_SBR_04,seawater metagenome,SBR
236,SBR_8.0,SBR,[to be added from GLOMICON partner's metadata],SBR,UDalhousie,8.0,GLOMICON_INTERCOMP_SBR_08,seawater metagenome,SBR
237,SBR_12.0,SBR,[to be added from GLOMICON partner's metadata],SBR,UDalhousie,12.0,GLOMICON_INTERCOMP_SBR_12,seawater metagenome,SBR
238,SBR_16.0,SBR,[to be added from GLOMICON partner's metadata],SBR,UDalhousie,16.0,GLOMICON_INTERCOMP_SBR_16,seawater metagenome,SBR


### Import Metadata files

In [3]:
#look at test file
file = '/Users/kpitz/github/GLOMICON/intercomparison/metadata/MBARI_NOAA_MIMARKS.survey.water.6.0_sharing.xlsx'
xl = pd.ExcelFile(file)
print('Sheets in excel file:',xl.sheet_names)  # see all sheet names
#sheets = xl.sheet_names
sheets = ['study_data', 'water_sample_data', 'amplicon_prep_data', 'analysis_data']
dfs= []
skiprow_num = [5,8,6,6]
for sheet, sr in zip(sheets, skiprow_num):
    print(sheet,sr)
    df = xl.parse(sheet,skiprows=sr)
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    df['sheet'] = sheet
    dfs.append(df)
df.head()

Sheets in excel file: ['README', 'study_data', 'water_sample_data', 'amplicon_prep_data', 'metag_prep_data', 'analysis_data', 'SRA_template', 'measurement_metadata', 'SRA Terms', 'Version log']
study_data 5
water_sample_data 8
amplicon_prep_data 6
analysis_data 6


/Users/kpitz/anaconda3/envs/scipy_test/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,amplicon_sequenced,trim_method,cluster_method,pid_clustering,taxa_class_method,taxa_ref_db,code_repo,date_modified,sheet
0,18S V9,atropos,dada2,ASV,BLAST; MEGAN LCA,genbank NR,https://github.com/MBARI-BOG/BOG-Banzai-Dada2-...,2024-08-15 08:44:22.928,analysis_data


In [4]:
def water_amplicon_merge(water_df, amplicon_df):
    df = water_df.set_index('sample_name')
    df = df.drop(['sheet', 'date_modified','modified_by'], axis=1)
    # add amplicon metadata to water sample metadata (some water samples don't have amplicon data if not sequenced in institute)
    df = df.join(amplicon_df.set_index('sample_name'), rsuffix='_amplicon_sheet')
    df = df.drop(['sheet', 'date_modified','modified_by'], axis=1)
    return df
def study_data_merge(study_meta, bioinf_meta, institute_name):
    df = pd.concat([study_meta, bioinf_meta], axis=1)
    df = df.drop(['sheet', 'date_modified'], axis=1)
    df = df.T
    df = df.rename(columns={0:institute_name})
    return df
    

In [5]:
# study_dfs = []
df = study_data_merge(dfs[0], dfs[3], 'MBARI')
study_dfs.append(df)
df.head()

,MBARI
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid...."
project_description,"Replicate filters, originating from surface wa..."


In [6]:
df = water_amplicon_merge(dfs[1], dfs[2])
df.head()

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,project_id,...,seq_meth,amplicon_sequenced,target_gene,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond
sample_name,,,,,,,,,,,,,,,,,,,,,
CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2
CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,Illumina MiSeq 2x150,18S V9,18S rRNA,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2


In [7]:
df = dfs[1].copy()
df.set_index('sample_name', inplace=True)
# add amplicon metadata to water sample metadata (some water samples don't have amplicon data if not sequenced in institute)
df = df.join(dfs[2].set_index('sample_name'), rsuffix='_amplicon_sheet')
print(list(df))

df.head()

['Collecting Institute', 'Analyzing Institute', 'cruise_id', 'station', 'ctd_bottle_no', 'sample_replicate', 'source_material_id', 'extract_number', 'sample_title', 'project_id', 'organism', 'collection_date_local', 'collection_date', 'depth', 'env_broad_scale', 'env_local_scale', 'env_medium', 'geo_loc_name', 'waterBody', 'countryCode', 'decimalLatitude', 'decimalLongitude', 'samp_vol_we_dna_ext', 'samp_collect_device', 'samp_mat_process', 'size_frac', 'sample_type', 'collection_method', 'basisOfRecord', 'chlorophyll', 'density', 'fluor', 'pressure', 'salinity', 'samp_store_temp', 'temp', 'date_modified', 'modified_by', 'sheet', 'library_id', 'title', 'library_strategy', 'library_source', 'library_selection', 'lib_layout', 'platform', 'instrument_model', 'filetype', 'filename', 'filename2', 'seq_facility', 'seq_meth', 'amplicon_sequenced', 'target_gene', 'target_subfragment', 'pcr_primer_forward', 'pcr_primer_reverse', 'pcr_primer_name_forward', 'pcr_primer_name_reverse', 'pcr_primer_

,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,project_id,...,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified_amplicon_sheet,modified_by_amplicon_sheet,sheet_amplicon_sheet
sample_name,,,,,,,,,,,,,,,,,,,,,
CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-08-15 08:42:28.469,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-06-18 19:31:14.358,kpitz@mbari.org,amplicon_prep_data
CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),GLOMICON,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data


In [8]:
# study-wide metadata can organize by institution?
df = pd.concat([dfs[3], dfs[0]], axis=1)
print(list(df))
df = df.drop(['sheet', 'date_modified'], axis=1)
df = df.T
df = df.rename(columns={0:'MBARI'})

df

['amplicon_sequenced', 'trim_method', 'cluster_method', 'pid_clustering', 'taxa_class_method', 'taxa_ref_db', 'code_repo', 'date_modified', 'sheet', 'project_id', 'project_name', 'project_id_external', 'project_contact', 'project_description', 'type', 'license', 'associated_parties', 'study_area_description', 'date_modified', 'sheet']


,MBARI
amplicon_sequenced,18S V9
trim_method,atropos
cluster_method,dada2
pid_clustering,ASV
taxa_class_method,BLAST; MEGAN LCA
taxa_ref_db,genbank NR
code_repo,https://github.com/MBARI-BOG/BOG-Banzai-Dada2-...
project_id,GLOMICON_INTERCOMP
project_name,Intercomparison of filtered seawater samples a...
project_id_external,GLOMICON_INTERCOMP


In [9]:
dfs[0]

,project_id,project_name,project_id_external,project_contact,project_description,type,license,associated_parties,study_area_description,date_modified,sheet
0,GLOMICON_INTERCOMP,Intercomparison of filtered seawater samples a...,GLOMICON_INTERCOMP,"Kathleen Pitz, kpitz@mbari.org, https://orcid....","Replicate filters, originating from surface wa...",Occurrence,CC-BY 4.0,"Francisco Chavez, chfr@mbari.org",eDNA from north of Monterey Bay collected duri...,2024-08-22 14:14:36.023,study_data


In [10]:
# Join two sample_name dfs:
#df = pd.concat([dfs[1].set_index('sample_name'), dfs[2].set_index('sample_name')], axis=0)
df = dfs[1].copy()
df['dup'] = df.duplicated('sample_name', keep=False)
df = df.loc[df['source_material_id'].isna()==False]
#df = df.dropna('source_material_id')
df = df.sort_values('dup')

df

,sample_name,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,...,density,fluor,pressure,salinity,samp_store_temp,temp,date_modified,modified_by,sheet,dup
0,CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-07-31 12:16:48.569,NaN,water_sample_data,False
1,CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data,False
2,CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data,False
3,CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:05:42.928,kpitz@mbari.org,water_sample_data,False
4,CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data,False
45,NaN,MBARI,NaN,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy23,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,True
44,NaN,MBARI,NaN,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy21,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,True
43,NaN,MBARI,SBR,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy20,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,True
42,NaN,MBARI,AWI,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy19,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,True
41,NaN,MBARI,NaN,CN18S,TS,12.0,NaN,CN18Sc37_12_Rep_Stdy17,NaN,Seawater sample Monterey Bay (GLOMICON),...,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,NaT,NaN,water_sample_data,True


In [11]:
dfs[2].head()

,sample_name,library_id,title,library_strategy,library_source,library_selection,lib_layout,platform,instrument_model,filetype,...,target_subfragment,pcr_primer_forward,pcr_primer_reverse,pcr_primer_name_forward,pcr_primer_name_reverse,pcr_primer_reference,pcr_cond,date_modified,modified_by,sheet
0,CN18Sc37_12_Rep_Stdy5_AO,AO9,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-08-15 08:42:28.469,NaN,amplicon_prep_data
1,CN18Sc37_12_Rep_Stdy10_AO,AO10,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,2024-06-18 19:31:14.358,kpitz@mbari.org,amplicon_prep_data
2,CN18Sc37_12_Rep_Stdy14_AO,AO11,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
3,CN18Sc37_12_Rep_Stdy18_AO,AO12,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data
4,CN18Sc37_12_Rep_Stdy22_AO,AO13,18S amplicon metabarcoding GLOMICON comparison...,AMPLICON,METAGENOMIC,PCR,paired,ILLUMINA,Illumina MiSeq,fastq,...,V9,GTACACACCGCCCGTC,TGATCCTTCTGCAGGTTCACCTAC,Euk_1391f,EukBr,"Amaral-Zettler et al. 2009, Stoeck et al. 2010...",dx.doi.org/10.17504/protocols.io.36wgq3d8olk5/v2,NaT,NaN,amplicon_prep_data


In [12]:
dfs[1].head()

,sample_name,Collecting Institute,Analyzing Institute,cruise_id,station,ctd_bottle_no,sample_replicate,source_material_id,extract_number,sample_title,...,chlorophyll,density,fluor,pressure,salinity,samp_store_temp,temp,date_modified,modified_by,sheet
0,CN18Sc37_12_Rep_Stdy5_AO,MBARI,MBARI,CN18S,TS,12.0,5.0,CN18Sc37_12_Rep_Stdy5,AO9,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-07-31 12:16:48.569,NaN,water_sample_data
1,CN18Sc37_12_Rep_Stdy10_AO,MBARI,MBARI,CN18S,TS,12.0,10.0,CN18Sc37_12_Rep_Stdy10,AO10,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data
2,CN18Sc37_12_Rep_Stdy14_AO,MBARI,MBARI,CN18S,TS,12.0,14.0,CN18Sc37_12_Rep_Stdy14,AO11,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data
3,CN18Sc37_12_Rep_Stdy18_AO,MBARI,MBARI,CN18S,TS,12.0,18.0,CN18Sc37_12_Rep_Stdy18,AO12,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:05:42.928,kpitz@mbari.org,water_sample_data
4,CN18Sc37_12_Rep_Stdy22_AO,MBARI,MBARI,CN18S,TS,12.0,22.0,CN18Sc37_12_Rep_Stdy22,AO13,Seawater sample Monterey Bay (GLOMICON),...,8.1461,25.328,0.3458,1.532,33.5385,-80.0,12.6583 °C,2024-06-18 19:04:45.094,kpitz@mbari.org,water_sample_data


In [13]:
df = pd.concat(dfs, axis=0)
df = df.dropna(how='all', axis=1)
df = df.dropna(how='all', axis=0)
df = df.T
df

,0,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,0
project_id,GLOMICON_INTERCOMP,GLOMICON,GLOMICON,GLOMICON,GLOMICON,GLOMICON,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_name,Intercomparison of filtered seawater samples a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_id_external,GLOMICON_INTERCOMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_contact,"Kathleen Pitz, kpitz@mbari.org, https://orcid....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
project_description,"Replicate filters, originating from surface wa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cluster_method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dada2
pid_clustering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASV
taxa_class_method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BLAST; MEGAN LCA
taxa_ref_db,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,genbank NR
